In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
#读取数据
df = pd.read_csv('train.csv',header=0) #读取训练数据
train_data = df.sample(frac=1) #打乱训练数据
test_data = pd.read_csv('test.csv',header=0) #读取测试数据

In [3]:
#每个批次的大小
batch_size = 500
#计算一共有多少个批次
n_batch = train_data.shape[0] // batch_size
#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) #平均值
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev) #标准差
        tf.summary.scalar('max',tf.reduce_max(var)) #最大值
        tf.summary.scalar('min',tf.reduce_min(var)) #最小值
        tf.summary.histogram('histogram',var) #直方图
#初始化权值
def weight_variables(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1) #生成一个截断的正态分布
    name = name
    return tf.Variable(initial,name)
#初始化偏置
def bias_variable(shape,name):
    initial = tf.constant(0.1,shape=shape)
    name = name
    return tf.Variable(initial,name)
#前向传播
def inference(input_tensor,regularizer):
    #第1层
    with tf.name_scope('Layer1'):
        w1 = weight_variables([115,500],name = 'W1')
        variable_summaries(w1) #分析第1层的权重变化
        b1 = bias_variable([500],name = 'b1') 
        variable_summaries(b1) #分析第1层的偏置变化
        tf.add_to_collection('losses', regularizer(w1))
        layer1 = tf.nn.tanh(tf.matmul(x,w1)+b1)
        layer1_drop = tf.nn.dropout(layer1,keep_prob)
    #第2层
    with tf.name_scope('Layer2'):
        w2 = weight_variables([500,450],name = 'W2')
        variable_summaries(w2) #分析第2层的权重变化
        b2 = bias_variable([450],name = 'b2') 
        variable_summaries(b2) #分析第2层的偏置变化
        tf.add_to_collection('losses', regularizer(w2))
        layer2 = tf.nn.tanh(tf.matmul(layer1_drop,w2)+b2)
        layer2_drop = tf.nn.dropout(layer2,keep_prob)
    #第3层
    with tf.name_scope('Layer3'):
        w3 = weight_variables([450,400],name = 'W3')
        variable_summaries(w3) #分析第3层的权重变化
        b3 = bias_variable([400],name = 'b3') 
        variable_summaries(b3) #分析第3层的偏置变化
        tf.add_to_collection('losses', regularizer(w3))
        layer3 = tf.nn.tanh(tf.matmul(layer2_drop,w3)+b3)
        layer3_drop = tf.nn.dropout(layer3,keep_prob)
    #第4层
    with tf.name_scope('Layer4'):
        w4 = weight_variables([400,300],name = 'W4')
        variable_summaries(w4) #分析第4层的权重变化
        b4 = bias_variable([300],name = 'b4') 
        variable_summaries(b3) #分析第4层的偏置变化
        tf.add_to_collection('losses', regularizer(w4))
        layer4 = tf.nn.tanh(tf.matmul(layer3_drop,w4)+b4)
        layer4_drop = tf.nn.dropout(layer4,keep_prob)
    #第5层
    with tf.name_scope('Layer5'):
        w5 = weight_variables([300,200],name = 'W5')
        variable_summaries(w5) #分析第5层的权重变化
        b5 = bias_variable([200],name = 'b5') 
        variable_summaries(b5) #分析第5层的偏置变化
        tf.add_to_collection('losses', regularizer(w5))
        layer5 = tf.nn.tanh(tf.matmul(layer4_drop,w5)+b5)
        layer5_drop = tf.nn.dropout(layer5,keep_prob)
    #第6层
    with tf.name_scope('Layer6'):
        w6 = weight_variables([200,100],name = 'W6')
        variable_summaries(w6) #分析第6层的权重变化
        b6 = bias_variable([100],name = 'b6') 
        variable_summaries(b6) #分析第6层的偏置变化
        tf.add_to_collection('losses', regularizer(w6))
        layer6 = tf.nn.relu(tf.matmul(layer5_drop,w6)+b6)
        layer6_drop = tf.nn.dropout(layer6,keep_prob)
    #第7层
    with tf.name_scope('Layer7'):
        w7 = weight_variables([100,100],name = 'W7')
        variable_summaries(w7) #分析第7层的权重变化
        b7 = bias_variable([100],name = 'b7') 
        variable_summaries(b7) #分析第7层的偏置变化
        tf.add_to_collection('losses', regularizer(w7))
        layer7 = tf.nn.relu(tf.matmul(layer6_drop,w7)+b7)
        layer7_drop = tf.nn.dropout(layer7,keep_prob)
    #第8层
    with tf.name_scope('Layer8'):
        w8 = weight_variables([100,11],name = 'W8')
        variable_summaries(w8) #分析第8层的权重变化
        b8 = bias_variable([11],name = 'b8') 
        variable_summaries(b8) #分析第8层的偏置变化
        tf.add_to_collection('losses', regularizer(w8))
        prediction = tf.nn.softmax(tf.matmul(layer7_drop,w8)+b8) 
    return prediction

with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32,[None,115],name = 'x-input')
    y = tf.placeholder(tf.float32,[None,11],name = 'y-input')
    keep_prob = tf.placeholder(tf.float32)
    
regularizer = tf.contrib.layers.l2_regularizer(0.001) #L2正则化
prediction = inference(x,regularizer)  #预测输出
global_step = tf.Variable(0,trainable=False) #存储训练轮数，设置为不可训练  

#定义损失函数
with tf.name_scope('Loss'):
    cross_entropy_mean  = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    #总损失等于交叉熵损失和正则化损失的和
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses')) 
    tf.summary.scalar('loss',loss) #分析误差的变化

#设置滑动平均方法
with tf.name_scope('Variable_Averages'):
    #定义滑动平均类
    variable_averages = tf.train.ExponentialMovingAverage(0.99, global_step)  
    #在所有可训练的变量上使用滑动平均
    variable_averages_op = variable_averages.apply(tf.trainable_variables()) 
    
with tf.name_scope('Training'):
    #设置指数衰减法
    learning_rate = tf.train.exponential_decay(0.5,global_step,1500,0.99) 
    #采用梯度下降法优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    #同时反向传播和滑动平均
    train_op = tf.group(train_step, variable_averages_op) 

#求准确率
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) #结果存放在一个布尔型列表中（true,false）
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #cast表示类型转换,转换为(1,0)
    tf.summary.scalar('accuracy',accuracy)

merged = tf.summary.merge_all() #合并所有的summary
init = tf.global_variables_initializer() #初始化变量
saver = tf.train.Saver() #保存模型
#启动图
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs1/',sess.graph)
    for epoch in range(50):
        for batch in range(n_batch):
            a = batch * batch_size
            b = (batch + 1) * batch_size
            batch_xs = train_data.iloc[a:b,:-1]
            batch_ys = sess.run(tf.one_hot(train_data.iloc[a:b,-1],11))
            summary,_,l = sess.run([merged,train_op,loss],feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.6})
        print(l)
        writer.add_summary(summary,epoch) #写入日志文件中    
        train_label = sess.run(tf.one_hot(train_data.iloc[:,-1],11))
        test_label = sess.run(tf.one_hot(test_data.iloc[:,-1],11))
        train_acc = sess.run(accuracy,feed_dict={x:train_data.iloc[:,:-1],y:train_label,keep_prob:1.0})#训练集
        test_acc = sess.run(accuracy,feed_dict={x:test_data.iloc[:,:-1],y:test_label,keep_prob:1.0}) #测试集
        print("Iter " + str(epoch) + ", Training Accuracy = " + str(train_acc)+ ", Testing Accuracy = " + str(test_acc))
    saver.save(sess,'model/my_model1.ckpt')

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

3.8647554
Iter 0, Training Accuracy = 0.7949194, Testing Accuracy = 0.67848235
3.3518836
Iter 1, Training Accuracy = 0.84538966, Testing Accuracy = 0.71612126
3.0283794
Iter 2, Training Accuracy = 0.88228923, Testing Accuracy = 0.764039
2.730896
Iter 3, Training Accuracy = 0.90355223, Testing Accuracy = 0.77764344
2.4837203
Iter 4, Training Accuracy = 0.8720272, Testing Accuracy = 0.75323105
2.3169315
Iter 5, Training Accuracy = 0.90868324, Testing Accuracy = 0.79532915
2.181059
Iter 6, Training Accuracy = 0.91090024, Testing Accuracy = 0.7870153
2.080475
Iter 7, Training Accuracy = 0.90306515, Testing Accuracy = 0.78384095
1.9972873
Iter 8, Training Accuracy = 0.90859926, Testing Accuracy = 0.7897362
1.9388039
Iter 9, Training 